In [1]:
import sys
import os
sys.path.append("../data-generator")
sys.path.append("../info-retrieval")
import contriever.contriever_final # Asmita's contriever

/home/kastan/utils/miniconda3/envs/nlp_v2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
USER_QUESTION = 'What are the inputs and outputs of a Gray code counter?'
NUM_ANSWERS_GENERATED = 5

## Using Asmita's Contriever

In [3]:
my_contriever = contriever.contriever_final.ContrieverCB()

In [4]:
# my_contriever.generate_embeddings("../data-generator/prompt engineering/gpt-3 semantic search/1_top_quality.json")
contriever_contexts = my_contriever.retrieve_topk("What are the inputs and outputs of a Gray code counter?", path_to_json = "../data-generator/split_textbook/paragraphs.json", k = NUM_ANSWERS_GENERATED)

top_context_list = list(contriever_contexts.values())
for i, context in enumerate(top_context_list):
    res = []
    for sub in context:
        res.append(sub.replace("\n", ""))
    top_context_list[i] = "".join(res)
print('\n\n'.join(top_context_list))

Let's begin with a two-bit Gray code counter with no inputs.As we mentioned in Notes Set 2.1, a Gray code is a cycle over allbit patterns of a certain length in which consecutive patterns differin exactly one bit.

A fully-specified transition diagram for a two-bit Gray code counter appears below.With no inputs, the states simply form a loop, withthe counter moving from one state to the next each cycle.

We have now verified the implementation.

Now we'll add a third bit to our counter, but again use a Gray codeas the basis for the state sequence.

{{file=part3/figs/part3-min-components.eps,width=3.84in}}


### OPT

In [2]:
# clear cuda memory
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

/home/kastan/utils/miniconda3/envs/nlp_v2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0

In [6]:
from transformers import GPT2Tokenizer, OPTForCausalLM
import torch

model = OPTForCausalLM.from_pretrained("facebook/opt-1.3b") # opt-350m
tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-1.3b")
# model.to("cuda")

In [7]:
# Generate
response_list = []
for i in range(NUM_ANSWERS_GENERATED):
  prompt = "Please answer this person's question accurately, clearly and concicely. Context: " + top_context_list[i] + '\n' + "Question: " + USER_QUESTION + '\n' + "Answer: "
  inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
  model = model.to("cuda")
  
  generate_ids = model.generate(inputs.input_ids, max_length=345, do_sample=True, top_k=50, top_p=0.95, temperature=0.95, num_return_sequences=1, repetition_penalty=1.2, length_penalty=1.2, pad_token_id=tokenizer.eos_token_id)
  response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  opt_answer = response.split("Answer:")[1]
  response_list.append(opt_answer)
  
print('\n---------------------------------NEXT---------------------------------\n'.join(response_list))

  Inputs
Interspersed with other values
Outputs
The following example shows the input structure (and its output) for a typical Gray Code Counter.It should be noted that some Gray codes do not allow for simple addition or subtraction of multiple bits; instead they require the use of a special value called a 1-to-5 accumulator, where five 0s can only sum to 4 1s.For simplicity, we will assume that these "special" functions are present every time the Gray code counter changes its state.We also assume a binary count as an extra level of abstraction when describing the function.This means there are three levels of indirection between inputs and actual output:
---------------------------------NEXT---------------------------------
   Inputs: 1 = (0, 1) = 0; 2 = (1, 2) = 5 ; 3 = 4 = 8 , etc.... Output : A - Gray code Counter Output is simply an A / B bit binary value, which determines the current stateof the counters. An output of zero would indicate that thereis no change in the counter, whil

In [8]:
print('\n---------------------------------NEXT---------------------------------\n'.join(response_list))

  Inputs
Interspersed with other values
Outputs
The following example shows the input structure (and its output) for a typical Gray Code Counter.It should be noted that some Gray codes do not allow for simple addition or subtraction of multiple bits; instead they require the use of a special value called a 1-to-5 accumulator, where five 0s can only sum to 4 1s.For simplicity, we will assume that these "special" functions are present every time the Gray code counter changes its state.We also assume a binary count as an extra level of abstraction when describing the function.This means there are three levels of indirection between inputs and actual output:
---------------------------------NEXT---------------------------------
   Inputs: 1 = (0, 1) = 0; 2 = (1, 2) = 5 ; 3 = 4 = 8 , etc.... Output : A - Gray code Counter Output is simply an A / B bit binary value, which determines the current stateof the counters. An output of zero would indicate that thereis no change in the counter, whil

### ReRanking (MS-Marco Cross Encoder)

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/ms-marco-MiniLM-L-6-v2')
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/ms-marco-MiniLM-L-6-v2')

assert len([USER_QUESTION] * NUM_ANSWERS_GENERATED ) == len(response_list)

features = tokenizer([USER_QUESTION] * NUM_ANSWERS_GENERATED, response_list,  padding=True, truncation=True, return_tensors="pt")

model.eval()
with torch.no_grad():
    scores = model(**features).logits
    print(scores)

tensor([[ 7.8039],
        [ 8.3896],
        [-4.4525],
        [ 4.2606],
        [ 0.4363]])


#### Todo: Using Jerome's Doc-Query
(lots of dependencies)

# Try running this 👇 This works running our main

In [ ]:
import torch
import main

ta = main.TA_Pipeline()

USER_QUESTION = "user_question = 'What are the inputs and outputs of a Gray code counter?'"
NUM_ANSWERS_GENERATED = 5

top_context_list = ta.contriever(user_question=USER_QUESTION, num_answers_generated=NUM_ANSWERS_GENERATED)
generated_answers_list = ta.OPT(top_context_list)
scores = ta.re_ranking_ms_marco(generated_answers_list)
index_of_best_answer = torch.argmax(scores) # get best answer
print("Best answer 👇\n", generated_answers_list[index_of_best_answer])